# 1.1 - Explore forest-masked GBIF data

In [11]:
import os
from dotenv import find_dotenv, load_dotenv
import geopandas as gpd
from src.conf.parse_params import config as cfg
from src.utils.df_utils import read_gdf

load_dotenv(find_dotenv())
os.chdir(os.environ["PROJECT_ROOT"])

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## How many species in total?

Load the dataframe

In [31]:
gbif = read_gdf(cfg["gbif"]["masked"]).astype({"species": "string[pyarrow]"})

In [34]:
gbif.species.value_counts()

species
Fagus sylvatica            147298
Quercus robur              104129
Sorbus aucuparia            97017
Urtica dioica               92983
Picea abies                 79606
                            ...  
Taraxacum tortuosum             1
Taraxacum insigne               1
Taraxacum lucescens             1
Taraxacum zevenbergenii         1
Taraxacum replicatum            1
Name: count, Length: 9219, dtype: int64[pyarrow]

It looks like _Fagus sylvatica_ (European beech) is the most abundant species, followed by _Quercus robur_ (pedunculate oak), and there are 9,219 unique species occurrences.

## Querying approach

We need to query species occurrences within a given radius at each grid cell center for each species.

To do this we will likely need to leverage an [R-tree](https://en.wikipedia.org/wiki/R-tree) for (relatively) quick spatial indexing. This can be done using the built-in `GeoDataFrame.sindex` method from `geopandas`.

We can then get a list of all unique species in the dataset, and for each species we can then iterate over each grid cell centroid and identify all occurrences of the respective species present in the query radius. 

A rough implementation might look something like this.

In [ ]:
import geopandas as gpd
import rasterio
import numpy as np
import pygeos
from shapely.geometry import Point
from geopandas import GeoSeries

# Load the reference raster
with rasterio.open('reference.tif') as src:
    meta = src.meta

# Load the point data
points = gpd.read_file('points.geojson')

# Convert the GeoDataFrame to a GeoSeries of pygeos.Geometry objects
points_pygeos = gpd.GeoSeries(points['geometry'].apply(pygeos.from_shapely))

# Create a spatial index
sindex = pygeos.STRtree(points_pygeos)

# Get the unique species
species = points['species'].unique()

# Create a 3D numpy array
counts = np.zeros((len(species), meta['height'], meta['width']), dtype=meta['dtype'])

# For each species
for sp_idx, sp in enumerate(species):
    # Filter the points
    points_sp = points[points['species'] == sp]

    # For each cell
    for j in range(meta['width']):
        for i in range(meta['height']):
            # Compute the cell center
            x, y = src.xy(i, j)
            center = Point(x, y)
            # Create a circular buffer around the cell center
            buffer = center.buffer(1000)  # 1000 meters = 1 km
            # Use the spatial index to find the points within the buffer
            precise_matches_index = sindex.query(buffer, predicate='intersects')
            precise_matches = points.iloc[precise_matches_index]
            # Compute the inverse distance weights
            weights = 1 / precise_matches.distance(center)
            # Sum the weights and store it in the numpy array
            counts[sp_idx, i, j] = weights.sum()

# Update the metadata to reflect the number of layers
meta.update(count=len(species))

# Create a new raster file
with rasterio.open('stacked.tif', 'w', **meta) as dst:
    dst.write(counts)